## Importing libraries
Installing fastai library, used in their [free MOOC](http://course18.fast.ai/ml.html).

In [1]:
#!pip install --upgrade pip
!pip install fastai==0.7.0 ## Based on Fast.ai ML course

    100% |████████████████████████████████| 122kB 3.8MB/s 
    100% |████████████████████████████████| 496.4MB 70kB/s 
  Found existing installation: torch 1.0.1.post2
    Uninstalling torch-1.0.1.post2:
      Successfully uninstalled torch-1.0.1.post2
  Found existing installation: fastai 1.0.51
    Uninstalling fastai-1.0.51:
      Successfully uninstalled fastai-1.0.51
You are using pip version 19.0.3, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [3]:
import numpy as np 
import pandas as pd
from IPython.display import display
from fastai.imports import *
from fastai.structured import *
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
import os
from pandas_summary import DataFrameSummary
from matplotlib import pyplot as plt
import math

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
import graphviz
import re

import shap
import eli5
from eli5.sklearn import PermutationImportance
from pdpbox import pdp, get_dataset, info_plots

import IPython
from IPython.display import display
print(os.listdir("../input/"))

['club-mahindra-submission', 'clubmahindrapvt']


## Reading in the datasets

In [4]:
train = pd.read_csv("../input/clubmahindrapvt/train.csv")
test = pd.read_csv("../input/clubmahindrapvt/test.csv")

In [5]:
train.describe(include='all')

,reservation_id,booking_date,checkin_date,checkout_date,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,memberid,cluster_code,reservationstatusid_code,resort_id,amount_spent_per_room_night_scaled
count,341424,341424,341424,341424,341424.000000,341424.000000,341424.000000,341424.000000,341424.000000,341424.000000,341424.000000,341424.000000,341424.000000,341310.000000,336660.000000,341424.000000,341424.000000,341424,341424.000000,341424,341424,341424,341424,341424.000000
unique,341424,1621,1532,1531,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,NaN,101327,6,4,32,NaN
top,78c412f85e3ae49eb7e2c691160b41de8ae71c2e6b080c...,02/09/16,08/10/16,21/10/18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,NaN,81c5295dd0cd56d704882cd28c7c815658ba5796165b7d...,F,A,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,NaN
freq,1,1353,460,491,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67618,NaN,50,120623,312481,35331,NaN
mean,NaN,NaN,NaN,NaN,1.902810,2.208738,3.275361,0.362696,210.288905,1.772960,2.193832,2.890321,3.738346,2.673862,7.260708,4.763303,3.193390,NaN,1.124209,NaN,NaN,NaN,NaN,7.720801
std,NaN,NaN,NaN,NaN,0.915311,0.823779,1.756062,0.758019,865.735577,0.750806,1.551451,0.804260,2.479054,0.885650,4.701283,3.103909,1.167824,NaN,0.329821,NaN,NaN,NaN,NaN,1.090109
min,NaN,NaN,NaN,NaN,1.000000,1.000000,0.000000,0.000000,45.000000,1.000000,0.000000,1.000000,-45.000000,1.000000,1.000000,1.000000,0.000000,NaN,1.000000,NaN,NaN,NaN,NaN,1.600397
25%,NaN,NaN,NaN,NaN,1.000000,2.000000,2.000000,0.000000,45.000000,1.000000,1.000000,2.000000,2.000000,2.000000,4.000000,2.000000,2.000000,NaN,1.000000,NaN,NaN,NaN,NaN,7.187260
50%,NaN,NaN,NaN,NaN,2.000000,2.000000,3.000000,0.000000,45.000000,2.000000,2.000000,3.000000,3.000000,3.000000,8.000000,5.000000,3.000000,NaN,1.000000,NaN,NaN,NaN,NaN,7.860814
75%,NaN,NaN,NaN,NaN,3.000000,3.000000,4.000000,0.000000,45.000000,2.000000,3.000000,3.000000,4.000000,3.000000,8.000000,7.000000,4.000000,NaN,1.000000,NaN,NaN,NaN,NaN,8.419144


### Extract all info from dates

In [6]:
#add_datepart(train, 'checkin_date')   ## Need to read as dates
#add_datepart(test, 'checkin_date')

## Converting categorical data, if any, to factors

In [7]:
train_cats(train)
apply_cats(test, train)

## Processing the training and test data 

In [8]:
df_trn, y_trn, nas = proc_df(train, 'amount_spent_per_room_night_scaled')
df_test, _, _ = proc_df(test, na_dict=nas)

## Splitting into training and validation set

In [9]:
train_X, val_X, train_y, val_y = train_test_split(df_trn, y_trn, test_size=0.33, random_state=42)

## Defining function to calculate the evaluation metric

In [10]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [100*rmse(m.predict(train_X), train_y), 100*rmse(m.predict(val_X), val_y),     ## 100*RMSE
                m.score(train_X, train_y), m.score(val_X, val_y)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

## Train the random forest model using a single tree first

In [11]:
m = RandomForestRegressor(n_estimators=1, max_depth=3, min_samples_leaf=5,  bootstrap=False, n_jobs=-1)
%time m.fit(train_X, train_y)

print_score(m)

CPU times: user 960 ms, sys: 68 ms, total: 1.03 s
Wall time: 1.06 s
[106.02018127289637, 106.47168236181463, 0.05185083184416506, 0.05064316296637117]


In [12]:
m = RandomForestRegressor(n_estimators=20, min_samples_leaf=3, max_features=0.5, n_jobs=-1, oob_score=True)
m.fit(train_X, train_y)
print_score(m)

[62.756131685413706, 104.39766188454627, 0.6677906015839707, 0.0872690150405947, 0.0210514671708234]


In [13]:
pred = m.predict(df_test)

In [14]:
print(os.listdir("../input/club-mahindra-submission"))
submission = pd.read_csv('../input/club-mahindra-submission/sample_submission.csv')

['sample_submission.csv']


In [15]:
submission['amount_spent_per_room_night_scaled'] = pred
submission.to_csv('rf_submission_v1.csv', index=False)